(https://www.kaggle.com/c/ashrae-energy-prediction/)

# ASRAE - Modeling

In [34]:
import numpy as np
import pandas as pd

import seaborn as sns
import plotly as plt
import plotly.express as px
import plotly.figure_factory as ff
import matplotlib.pyplot as plt

from math import sqrt

import glob

import pandas_profiling as ppf
import sweetviz as sv
import missingno as msno

import pprint
import datetime
from math import sqrt

from plotly.subplots import make_subplots
import plotly.graph_objects as go

from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from sklearn.model_selection import TimeSeriesSplit, train_test_split

from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.tsa.arima.model import ARIMA
import statsmodels.api as sm

from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ParameterGrid

from prophet import Prophet

import xgboost as xgb


import pmdarima as pm
from pmdarima import model_selection
from pmdarima import pipeline
from pmdarima import preprocessing as ppc
from pmdarima import arima
from pmdarima.arima import ndiffs
from pmdarima.arima import ADFTest

print("pmdarima version: %s" % pm.__version__)

#import tools as tl

sns.set(rc={'figure.figsize':(30,20)})

pd.options.display.float_format = '{:.2f}'.format
pd.set_option("precision", 2)

plt.rcParams.update({'font.size': 22})
sns.set(font_scale = 2)

import pprint
pp = pprint.PrettyPrinter(indent=4)

import warnings
warnings.filterwarnings('ignore')

pmdarima version: 1.8.2


In [35]:
print(xgb.__version__)

1.4.2


In [36]:
pd.options.plotting.backend = "matplotlib"

In [37]:
pwd

'G:\\My Drive\\Code\\Springboard_Capstone_Energy'

In [38]:
# Drop Specific columns
def building_meter_drop(df):
    return df.drop(columns=['precip_depth_1_hr','wind_direction','wind_speed','dew_temperature','year_built','floor_count','cloud_coverage'],axis=1)

In [39]:
# Interpolate Missing Data
def building_meter_interp(df):
    df['sea_level_pressure'] = df['sea_level_pressure'].interpolate(method='time')
    df['meter_reading'] = df['meter_reading'].interpolate(method='time')
    df['air_temperature'] = df['air_temperature'].interpolate(method='time')
    return df

In [40]:
# Aggregate remaining Data
def building_meter_agg(df,resamp_str):
    df_samp = df
    agg_dict={'meter_reading':'mean', 
                  'air_temperature':'max',
                  'building_id': 'max',
                  'site_id': 'max',
                  'square_feet': 'max',
                  'day_of_week':'max',
                  'weekend':'max',
                  'month':'max',
                  'season':'max'
                  }

    # Resample Dataframe
    df = df.resample(resamp_str).agg(agg_dict)
    
    return df

## See Statistics written for each file (building-meter) combo

In [41]:
# See Building Statistics for missing data
df_stats = pd.read_csv('.//data_clean//building_reading_stats.csv')
# Keep only data that has at least 50 % of it

In [42]:
df_stats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2375 entries, 0 to 2374
Data columns (total 19 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   file_name                     2375 non-null   object 
 1   building                      2375 non-null   int64  
 2   meter                         2375 non-null   int64  
 3   len_train                     2375 non-null   int64  
 4   len_zero                      2375 non-null   int64  
 5   per_data                      2375 non-null   float64
 6   category                      2375 non-null   object 
 7   corr_meter_to_airtemp         2375 non-null   float64
 8   corr_meter_to_airtemp_wknd    2373 non-null   float64
 9   corr_meter_to_airtemp_wkdy    2375 non-null   float64
 10  corr_meter_to_airtemp_winter  2325 non-null   float64
 11  corr_meter_to_airtemp_spring  2342 non-null   float64
 12  corr_meter_to_airtemp_summer  2339 non-null   float64
 13  cor

In [43]:
# Modeling Choices
# Buildings with 1
df_stats.sort_values(['building','meter'],axis=0,inplace=True)
df_stats.reset_index(drop=True,inplace=True)

In [44]:
# These represent buildings that are in top, mid and lower in square footage
# choose from these when we model our loop
lrg = [1148]
med = [1063, 1309, 991, 144, 693]
sm = [846, 621, 822, 816, 44]

# lrg = [869, 1148, 375, 365, 269]

bldgno = []
bldgno.extend(lrg)
bldgno.extend(med)
bldgno.extend(sm)

# 0: Electricity 1: chilledwater 2: steam 3: hotwater

In [45]:
df_stats[(df_stats['building'] == 1148) & df_stats['meter'] == 1]

,file_name,building,meter,len_train,len_zero,per_data,category,corr_meter_to_airtemp,corr_meter_to_airtemp_wknd,corr_meter_to_airtemp_wkdy,corr_meter_to_airtemp_winter,corr_meter_to_airtemp_spring,corr_meter_to_airtemp_summer,corr_meter_to_airtemp_fall,square_feet,site_id,floor_count,bldg_size,meter_name
1686,1148_1.pkl,1148,1,8784,0,100.00,Office,0.78,0.79,0.79,0.21,0.74,0.45,0.76,861524,13,-1,XXL,CWater


In [46]:
# Examine building choices
df_stats[df_stats['building'] == 1148]

,file_name,building,meter,len_train,len_zero,per_data,category,corr_meter_to_airtemp,corr_meter_to_airtemp_wknd,corr_meter_to_airtemp_wkdy,corr_meter_to_airtemp_winter,corr_meter_to_airtemp_spring,corr_meter_to_airtemp_summer,corr_meter_to_airtemp_fall,square_feet,site_id,floor_count,bldg_size,meter_name
1685,1148_0.pkl,1148,0,8663,0,100.00,Office,-0.02,0.15,-0.04,-0.20,-0.11,-0.08,-0.08,861524,13,-1,XXL,Elec
1686,1148_1.pkl,1148,1,8784,0,100.00,Office,0.78,0.79,0.79,0.21,0.74,0.45,0.76,861524,13,-1,XXL,CWater
1687,1148_2.pkl,1148,2,8784,2,99.98,Office,-0.88,-0.86,-0.88,-0.83,-0.85,-0.43,-0.44,861524,13,-1,XXL,Steam


### Build list of files to include

# Build 3 Models on Selected Buildings

In [ ]:
# Data is daily
# # data_sizes = [360] # Start with one year # Train on whole year

output_vis_folder = '.\\visualization\\models\\short_train\\'

# Control Variables that will run these specific models if True
ARIMA = True
FBProphet = True
xgboost = True

# Show Visualizations
quiet_mode = True

resample = True
backfill = 'mean'
exogen = True

thresh = 50

# Dictionary to hold all the results, will turn into a dataframe
stats_all = {}
stats_list = []

stats = {}

# For each file in the directory
direct = '.\\data_clean\\buildings_by_meter\\'

files = df_stats['file_name']
bl = df_stats['building']

bld_mtr = ['1073_2', # Elec
           '1073_1',
           '1073_0',
           '1099_2', # Education 
           '1130_0',
           '1048_0',
           '1156_0',
           '1156_2',
           '1140_1',
           '1169_1',
           '1178_0',
           '107_0',
           '1180_2',
           '119_0',
           '1198_2',
           '120_0',
           '109_0',
           '1138_0',
           '1079_2',
           '126_0',
           '1179_1', # Entertainment
           '1183_0',
           '1196_0',
           '743_0',
           '663_0', # Food
           '1105_0',
           '1105_2',
           '670_0',
           '1209_2',
           '1154_2', # Lodging Residential
           '1154_0',
           '134_0',
           '1184_0',
           '135_0',
           '1124_0',
           '1091_0'
           '1175_0', # Manufacturing
           '1171_0',
           '1081_0',
           '1145_2',
           '1148_1', # Office
           '1164_1',
           '1151_1',
           '1149_0',
           '1188_0',
           '141_0',
           '146_0',
           '1204_2',
           '1210_1',
           '1133_2',
           '155_0',
           '1164_1',
           '1109_2',
           '684_0',
           '1108_0',
           '1205_0',
           '1212_1',
           '1220_0',
           '1047_0',
           '678_0',
           '1104_0'
           '1052_2',
           '1034_0',
           '1197_0',
           '1118_0',
           '1199_0'
          ]

for num,file in enumerate(files):
    if file.split('.pkl')[0] not in bld_mtr:
        continue
    else:
        print('File: ' + file)
    
    # Temporary dictionary that represents a row
    stats = {}
    
    # Screen results
    if df_stats.iloc[num]['per_data'] < 80:
        stats['status'] = 'FAIL: Length < 80% data present'
        print('not enough data')
        continue

    try:
        df = pd.read_pickle(direct + file)
    except FileNotFoundError:
        continue
    
    print('Length on load: ' + str(len(df)))
    stats['Len_load'] = len(df)
    
    if len(df) < 20000:
        stats['status'] = 'FAIL: Length < 20k'
        continue

    # Load key variables in dictionary
    stats['building'] = df.iloc[0]['building_id']
    stats['square_feet'] = df.iloc[0]['square_feet']   
    stats['meter'] = bldg_meter = df.iloc[0]['meter']
    stats['file_name'] = file
 
    # Process Columns
    df = building_meter_drop(df) # Drop poorly correlated columns
    df = building_meter_interp(df) # Time series linear interpolation due to standard time deltas
    df = building_meter_agg(df,'1D') # aggregate remaining columns
    
    print('Length after agg: ' + str(len(df)))
    stats['Len_agg-clean'] = len(df)
    
    # Find out how many lingering NA columns are left
    b4_NA = len(df)
    df.dropna(how='any',inplace=True)
    tot = b4_NA - len(df)
    print('Dropped N/A cols: ' + str(tot))
    
    #exit if we drop nan rows
    if tot > 100:
        stats['status'] = 'FAIL: Drop over 100 Rows'
        continue
    
    print('Length after blank drop: ' + str(len(df)))
    stats['Len_blank'] = len(df)
    
    # Keep track of Models
    model_ctr = 1 # Initialize Model Counter
    model_prefixes = []

    # Split the whole set
    df_train = df.loc['2016-01-01':'2016-09-30']
    # Test 4 weeks
    df_test = df.loc['2016-10-01':'2016-10-31']
    
    # skip building if we dont have enough data
#     try:
#         if (len(df_train) / len(df_test)) < 2.5:
#             print('Not enough training data')
#         continue
#     except ZeroDivisionError as ze:
#         print('DF empty')
#         continue
      
    X_train = df_train.drop(labels=['meter_reading'],axis=1)
    X_train.fillna(method='ffill')

    X_test = df_test.drop(labels=['meter_reading'],axis=1)
    X_test.fillna(method='ffill')

    y_train = df_train['meter_reading']
    y_train.fillna(method='ffill')

    y_test = df_test['meter_reading']
    y_test.fillna(method='ffill')
    
    # Get rid of constants
    X_train.drop(labels=['site_id','building_id','square_feet'],axis=1,inplace=True)
    X_test.drop(labels=['site_id','building_id','square_feet'],axis=1,inplace=True)
    
    #   ######################################    ARIMA   ##################################################
    if ARIMA:
#                 # Keep track of model prefix column names by building prefix to number iterations
        model_prefix = 'Model_' + str(model_ctr)
        model_prefixes.append(model_prefix)
        
        # Populate DataFrame with statistics
        stats[model_prefix + '_' + 'Type'] = 'ARIMA_Exogen'
        stats[model_prefix + '_' + 'Description'] = 'ARIMA_Exogen, with daily aggregation'
        stats[model_prefix + '_' + 'Len_Train'] = len(X_train)
        stats[model_prefix + '_' + 'Len_Test'] = len(X_test)

        print('ARIMA Model......',end=' ')

        # Try building model
        try:
            model = pm.auto_arima(y_train,X=X_train,trace=False,n_fits=10,seasonal=True,m=7,error_action='ignore')
            stats[model_prefix + '_' + 'Status'] = 'PASS'
        except ValueError as ve:
            print('Model will not converge')
            stats[model_prefix + '_' + 'Status'] = 'FAIL'

        preds, conf_int = model.predict(X=X_test,n_periods=y_test.shape[0], return_conf_int=True)

        predict = pd.Series(preds,index=y_test.index)

        # Keep each slice data here
        df_result = pd.concat([predict,y_test],axis=1)
        df_result.columns = ['meter_predict_ARIMA','meter_actual']

        MAPE_error = (mean_absolute_percentage_error(df_result['meter_predict_ARIMA'],df_result['meter_actual']))

        stats[model_prefix + '_' + 'MAPE'] = MAPE_error
        
        RMSE_error = (mean_squared_error(df_result['meter_predict_ARIMA'],df_result['meter_actual'],squared=False))
        stats[model_prefix + '_' + 'RMSE'] = RMSE_error
        
          # RMSE relative to the range of the test range
        stats[model_prefix + '_' + 'y_pred_RMSE_pct_Range'] = RMSE_error / \
                                             ((np.max(df_result['meter_actual'])) \
                                              - (np.min(df_result['meter_actual']))) \
                                              * 100


        # Create Graph
        plt.ioff()
        plt.figure(figsize=(20,10))
        plt.plot(df_result['meter_predict_ARIMA'],label="ARIMA")
        plt.plot(df_result['meter_actual'],label="Actual")
        plt.legend(loc = 'upper right')      
        plt.title('Bldg ' + str(stats['building']) + ' Meter ' + str(stats['meter']) + ' ' + model_prefix + '_' + 'ARIMA_Exogen')
        plt.xlabel('Date')
        plt.ylabel('Meter Reading')
        
        plt.savefig(output_vis_folder + str(stats['building']) + '_' + str(stats['meter']) + '_ARIMA_Exogen.jpg')
        print('MAPE: ' + str(MAPE_error)) 
          
         ######################################    PROPHET   ##################################################
        if FBProphet:           
            print('Prophet......',end=' ')
            
          
            params_grid = {'seasonality_mode':('multiplicative','additive'),
                   'changepoint_prior_scale':[0.1,0.5],
              'n_changepoints' : [100,200]}

            grid = ParameterGrid(params_grid)
            print('Tuning Prophet....')
            
            model_ctr += 1 # Increment Model Counter
            
            model_prefix = 'Model_' + str(model_ctr) # Build string for keys
            model_prefixes.append(model_prefix)
          

            # Need renaming for Prophet - data prep
            df_prophet_train = df_train.reset_index()
            df_prophet_test = df_test.reset_index()
            
            
            # Get rid of constants
            df_prophet_train.drop(labels=['site_id','building_id','square_feet'],axis=1,inplace=True)
            df_prophet_test.drop(labels=['site_id','building_id','square_feet'],axis=1,inplace=True)

            # Timestamp must be ds column
            df_prophet_train = df_prophet_train.rename(columns={'timestamp':'ds','meter_reading':'y'})
            df_prophet_test = df_prophet_test.rename(columns={'timestamp':'ds','meter_reading':'y'})
            
            
            # Populate DataFrame with statistics
            stats[model_prefix + '_' + 'Type'] = 'Prophet_Exogen'      
            stats[model_prefix + '_' + 'Len_Train'] = len(df_prophet_train)
            stats[model_prefix + '_' + 'Len_Test'] = len(df_prophet_test)

            MAPEs = []
            params = []

            tuning_iter = 0
            for val in grid:
                print('Tuning Iteration: ' + str(tuning_iter))
                # Initialize Prophet Model
                m = Prophet(daily_seasonality=True,
                           n_changepoints = val['n_changepoints'],
                           seasonality_mode = val['seasonality_mode'],
                           changepoint_prior_scale = val['changepoint_prior_scale'])
                m.add_regressor('air_temperature')
                m.add_regressor('day_of_week')
                m.add_regressor('weekend')
                m.add_regressor('month')
                m.add_regressor('season')
                m.fit(df_prophet_train)
                predicts = m.predict(df_prophet_test)

                MAPE_error = (mean_absolute_percentage_error(predicts['yhat'],df_prophet_test['y']))
                MAPEs.append(MAPE_error)
                tuning_iter += 1
                

            min_value = min(MAPEs)
            min_index = MAPEs.index(min_value)
            MAPE_error = MAPEs[min_index]
            best_parameter = grid[min_index]
            print(best_parameter)

            stats[model_prefix + '_' + 'MAPE'] = MAPE_error
            
                    
            RMSE_error = (mean_squared_error(predicts['yhat'],df_prophet_test['y'],squared=False))
            stats[model_prefix + '_' + 'RMSE'] = RMSE_error
        
              # RMSE relative to the range of the test range
            stats[model_prefix + '_' + 'y_pred_RMSE_pct_Range'] = RMSE_error / \
                                                 ((np.max(df_prophet_test['y'])) \
                                                  - (np.min(df_prophet_test['y']))) \
                                                  * 100

            plt.ioff()
            plt.figure(figsize=(20,10))
            plt.plot(predicts['yhat'],label="Prophet")
            plt.plot(df_prophet_test['y'],label="Actual")
            plt.legend(loc = 'upper right')
            
            plt.title('Bldg ' + str(stats['building']) + ' Meter ' + str(stats['meter']) + ' ' + \
                      model_prefix + '_' + 'Prophet_Exogen')
            plt.xlabel('Date')
            plt.ylabel('Meter Reading')
            
            plt.savefig(output_vis_folder + str(stats['building']) + '_' + str(stats['meter']) + '_Prophet_Exogen.jpg')

            print('Best MAPE: ' + str(MAPE_error))
                    
#             ######################################    XGBOOST   ##################################################
            if xgboost:
                print('XGBoost......',end=' ')
            
            

                model_ctr += 1
                model_prefix = 'Model_' + str(model_ctr)
                model_prefixes.append(model_prefix)

                # Populate DataFrame with statistics
                stats[model_prefix + '_' + 'Type'] = 'XGBoost'
                stats[model_prefix + '_' + 'Description'] = 'XGBoost_Exogen, with daily aggregation'
                stats[model_prefix + '_' + 'Len_Train'] = len(X_train)
                stats[model_prefix + '_' + 'Len_Test'] = len(X_test)
    
    
                params_grid = {# Parameters that we are going to tune.
                    'learning_rate': [0.01,0.1],
                    'max_depth': [3,10],
                    'min_child_weight': [1,5],
                    'subsample': [0.5,0.7],
                    'colsample_bytree': [0.5,0.7],
                    'n_estimators' : [100,500],
                    'objective': ['reg:squarederror'],
                }
                
                num_boost_round = 30
                
                num_boost_round = ''
                early_stopping_rounds = ''

                # Instantiate Model
                gbm_model = xgb.XGBRegressor()
                gsearch = GridSearchCV(estimator = gbm_model,
                                      param_grid = params_grid,
                                      scoring = 'mean_absolute_percentage_error',
                                      cv=5,
                                      n_jobs = -1,
                                      verbose = 1)

#                 print(gsearch.best_params_)

                gbm_model.fit(X_train,y_train)
                predict = gbm_model.predict(X_test)

                predict = pd.Series(predict,index=y_test.index)

                df_result = pd.concat([predict,y_test],axis=1,ignore_index=True)
                df_result.columns = ['meter_predict_XGBoost','meter_actual']
                df_result.sort_index(inplace=True)

                MAPE_error = (mean_absolute_percentage_error(predict,y_test))
                MAPEs.append(MAPE_error)
                tuning_iter += 1
                    
                stats[model_prefix + '_' + 'MAPE'] = MAPE_error
                
                plt.figure(figsize=(20,10))
                plt.plot(df_result['meter_predict_XGBoost'],label="XGBoost")
                plt.plot(df_result['meter_actual'],label="Actual")
                plt.legend(loc = 'upper right')
                plt.title('Bldg ' + str(stats['building']) + ' Meter ' + str(stats['meter']) + ' ' + model_prefix + '_' + 'XGBoost')
                plt.xlabel('Date')
                plt.ylabel('Meter Reading')
                plt.savefig(output_vis_folder + str(stats['building']) + '_' + str(stats['meter']) + '_XGBoost.jpg')
                print('MAPE: ' + str(MAPE_error))
                
                RMSE_error = (mean_squared_error(predict,y_test,squared=False))
                
                stats[model_prefix + '_' + 'RMSE'] = RMSE_error
                  # RMSE relative to the range of the test range
                stats[model_prefix + '_' + 'y_pred_RMSE_pct_Range'] = RMSE_error / \
                                                     ((np.max(df_result['meter_actual'])) \
                                                      - (np.min(df_result['meter_actual']))) \
                                                      * 100
                
#             df_viz = pd.DataFrame(df_result['meter_predict_XGBoost'],df_result['meter_predict_ARIMA'],df_result['meter_predict_Prophet'],index=df_result.index)
           

#             plt.plot(df_result['meter_predict_ARIMA'],label="ARIMA")
#             sns.lineplot(data=[df_result['meter_predict_XGBoost'],
#                                 df_result['meter_predict_ARIMA']],
#                          x=df_result.index,
#                          y=)
#             plt.plot(df_result['meter_predict_XGBoost'],label="XGBoost")
#             plt.plot(df_result['meter_actual'],label="Actual")
#             plt.legend(loc = 'upper right')
#             plt.savefig(output_vis_folder + str(stats['building']) + '_' + str(stats['meter']) + '_All.jpg')

#             print(df_viz)

        print()
        print()
    
        stats_list.append(stats)
        # Write dictionary and file every 10 buildings to save results
        if stats['building'] % 10 == 0:
            df_results = pd.DataFrame(stats_list)
            df_results = pd.DataFrame(stats_list)
            df_results.to_csv('Model_Results_1.csv',float_format="{:,.2f}".format)
            
df_results = pd.DataFrame(stats_list)
df_results.to_csv('Model_Results_3_Tuning.csv',float_format="{:,.2f}".format)
df_stats = df_results.merge(df_stats,how='left',on=['file_name'])
df_stats.to_csv('AllResults_Tuned.csv',float_format="{:,.2f}".format)

File: 107_0.pkl
Length on load: 26304
Length after agg: 1096
Dropped N/A cols: 0
Length after blank drop: 1096
ARIMA Model...... 

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.


MAPE: 0.18744081759974943
Prophet...... Tuning Prophet....
Tuning Iteration: 0


INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.


Tuning Iteration: 1


...

Tuning Iteration: 5


INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.


Tuning Iteration: 6


INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.


Tuning Iteration: 7
{'seasonality_mode': 'additive', 'n_changepoints': 200, 'changepoint_prior_scale': 0.1}
Best MAPE: 0.06126621813127285
XGBoost...... MAPE: 0.10278830156162742


File: 119_0.pkl
Length on load: 26304
Length after agg: 1096
Dropped N/A cols: 0
Length after blank drop: 1096
ARIMA Model...... 

In [ ]:
df_results.filter(regex='RMSE')

In [ ]:
df_results.filter(regex='Model_')
df.to_csv('results.csv')

In [ ]:
df_stats = df_stats.merge(df_results,how='right',on=['building','meter'])

In [ ]:
df_stats

In [ ]:
df= df_results.filter(regex='RMSE')
df.head()

In [ ]:
df_stats = df_results.merge(df_stats,how='left',on=['file_name'])

In [ ]:
df_stats.to_csv('AllResults_2_subset.csv',float_format="{:,.2f}".format)

In [ ]:
df_results.filter(regex='RMSE')

In [ ]:
df_results.filter(regex='Model_')
df.to_csv('results.csv')

In [ ]:
df_stats = df_stats.merge(df_results,how='right',on=['building','meter'])

In [ ]:
df_stats

In [ ]:
df= df_results.filter(regex='RMSE')
df.head()

## Time Series Split Cross Validation

In [ ]:
tss =  TimeSeriesSplit(n_splits=4,gap=4,test_size=14,max_train_size=120) #too generic
rmse = []
count = 1

results = pd.DataFrame()


# Write function that rolls next pd.delta?

for train_index, test_index in tss.split(df_train): 
    
    cv_train, cv_test = df_train.iloc[train_index]['meter_reading'], df_train.iloc[test_index]['meter_reading']
    model = pm.auto_arima(cv_train,trace=True,n_fits=20)
    print(model.summary())
    
    # 
    predicts = model.predict(n_periods=14)
    true_values = cv_test.values
    error = ((sqrt(mean_squared_error(predicts,true_values))))
    model.plot_diagnostics()
    rmse.append(error)
    print(error)
print('RMSE')
print(np.mean(rmse))

# Rolling Forecast ARIMA
### Walk-forward validation

In [ ]:
model.plot(forecast)

In [ ]:

#             # Fill Nas
#             #Counts of null values 
#             # Backfill with means
#             if np.max(X_train.isnull().sum().sort_values(ascending=False)) > 0:
#                 stats[model_prefix + '_' + 'X_train_Backfill'] = backfill
#                 if backfill == 'mean':
#                     X_train.fillna(X_train.mean(), inplace=True)

#             # Backfill with mean
#             if np.max(X_test.isnull().sum().sort_values(ascending=False)) > 0:
#                 stats[model_prefix + '_' + 'X_test_Backfill'] = backfill
#                 if backfill == 'mean':
#                     X_test.fillna(X_test.mean(), inplace=True)
                    
#             # Backfill with mean
#             if np.max(y_test.isnull().sum().sort_values(ascending=False)) > 0:
#                 stats[model_prefix + '_' + 'X_test_Backfill'] = backfill
#                 if backfill == 'mean':
#                     y_test.fillna(X_test.mean(), inplace=True)
                    
#             # Backfill with mean
#             if np.max(y_train.isnull().sum().sort_values(ascending=False)) > 0:
#                 stats[model_prefix + '_' + 'X_test_Backfill'] = backfill
#                 if backfill == 'mean':
#                     y_train.fillna(X_train.mean(), inplace=True)